# Wood Beam Clips

In [85]:
from functools import partial
from itertools import product
from IPython.display import Markdown
import numpy as np
import pandas as pd
from wsweng.wood.dowels import wood_bolt

## Givens

In [86]:
diameters = [0.625, 0.75, 0.875, 1.0]  # Nominal diameter, inches
thicknesses = [3.00, 3.50, 5.125, 5.25, 5.50, 7.0]
bolt_counts = range(2, 7, 1)
spacings = [2.5, 3, 4]
clip_thickness = 1/4.0
species = "DFL"
steel = "A36"
make_bolt = partial(
    wood_bolt,
    full_diameter=False,
    main_material=species,
    side_material=steel,
    side_thickness=clip_thickness
)

### Generate design permutations

Find all permutations of diameter, thickness, N and double shear:

In [87]:
p_dia = product(diameters, thicknesses, bolt_counts, spacings, [True, False])
bolts = pd.DataFrame(data=p_dia, columns=["d", "t", "n", "s", "ds"])
bolts.sample(3)

,d,t,n,s,ds
342,0.75,7.0,4,2.5,True
701,1.00,7.0,3,4.0,False
592,1.00,3.5,5,4.0,True


### Calculate capacity

In [88]:
_z0 = [make_bolt(_d, _t, double_shear=_ds).Zv(theta=90.0)
               for _d, _t, _ds in zip(bolts["d"], bolts["t"], bolts["ds"])]
_c_spcg = np.clip(bolts["s"]/(4*bolts["d"]), 0, 1.0)
bolts["c_spcg"] = np.where(_c_spcg < 0.5, 0, _c_spcg)  # Zero capacity for less than min spacing.
bolts["z"] = bolts["n"]*_c_spcg*_z0
bolts.sample(3)

,d,t,n,s,ds,c_spcg,z
461,0.875,5.250,3,4.0,False,1.0,2637.521747
327,0.750,5.500,6,3.0,False,1.0,4111.559732
628,1.000,5.125,6,4.0,True,1.0,11630.405782


### Geometry

In [89]:
bolts["h_beam"] = (bolts["n"] - 1)*bolts["s"] + 2*4*bolts["d"]
bolts["h_clip"] = (bolts["n"] - 1)*bolts["s"] + 3.0
bolts["d_end"] = np.clip(1.5*bolts["d"], 1.25, np.inf)

bolts.sample(3)

,d,t,n,s,ds,c_spcg,z,h_beam,h_clip,d_end
480,0.875,5.5,2,2.5,True,0.714286,2524.601164,9.5,5.5,1.3125
217,0.750,3.5,3,2.5,False,0.833333,1413.798108,11.0,8.0,1.2500
398,0.875,3.5,3,3.0,True,0.857143,3175.040419,13.0,9.0,1.3125


## Schedules

In [97]:
full_sched = bolts.pivot(index=["ds", "s", "d", "n", "h_beam", "h_clip"], columns=["t"], values="z")
display(full_sched.head(3).round(1))

t                                 3.000   3.500   5.125   5.250   5.500  \
ds    s   d     n h_beam h_clip                                           
False 2.5 0.625 2 7.5    5.5      889.6  1004.2  1021.8  1021.8  1021.8   
                3 10.0   8.0     1334.4  1506.3  1532.7  1532.7  1532.7   
                4 12.5   10.5    1779.2  2008.5  2043.5  2043.5  2043.5   

t                                 7.000  
ds    s   d     n h_beam h_clip          
False 2.5 0.625 2 7.5    5.5     1021.8  
                3 10.0   8.0     1532.7  
                4 12.5   10.5    2043.5

In [112]:
def display_sched(d, s):
    _md = ""
    _md += f"#### {d:.3f}\"dia at {s:.2f}\"oc., Single Shear\n\n"
    sched = full_sched.loc[(False, s, d, slice(None))]
    _md += sched.round(1).reset_index(level=("h_beam", "h_clip")).to_markdown()

    _md += "\n\n"
    _md += f"#### {d:.3f}\"dia at {s:.2f}\"oc., Double Shear\n\n"
    sched = full_sched.loc[(True, s, d, slice(None))]
    _md += (sched.round(1).reset_index(level=("h_beam", "h_clip"))).to_markdown()

    _md = _md.replace("h_beam", "Min Beam Height")
    _md = _md.replace("h_clip", "Clip Length")

    return Markdown(_md)

### 3/4 Bolts, 3" Spacing

In [113]:
display_sched(0.75, 3)

#### 0.750"dia at 3.00"oc., Single Shear

|   n |   Min Beam Height |   Clip Length |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|----:|---------:|---------:|-------:|-------:|--------:|-------:|-------:|-------:|
|   2 |        9 |        6 | 1005.1 | 1131   |  1370.5 | 1370.5 | 1370.5 | 1370.5 |
|   3 |       12 |        9 | 1507.7 | 1696.6 |  2055.8 | 2055.8 | 2055.8 | 2055.8 |
|   4 |       15 |       12 | 2010.3 | 2262.1 |  2741   | 2741   | 2741   | 2741   |
|   5 |       18 |       15 | 2512.8 | 2827.6 |  3426.3 | 3426.3 | 3426.3 | 3426.3 |
|   6 |       21 |       18 | 3015.4 | 3393.1 |  4111.6 | 4111.6 | 4111.6 | 4111.6 |

#### 0.750"dia at 3.00"oc., Double Shear

|   n |   Min Beam Height |   Clip Length |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|----:|---------:|---------:|-------:|-------:|--------:|-------:|-------:|-------:|
|   2 |        9 |        6 | 1939.8 | 2263.1 |  2741   | 2741   | 2741   | 2741   |
|   3 |       12 |        9 | 2909.7 | 3394.6 |  4111.6 | 4111.6 | 4111.6 | 4111.6 |
|   4 |       15 |       12 | 3879.6 | 4526.2 |  5482.1 | 5482.1 | 5482.1 | 5482.1 |
|   5 |       18 |       15 | 4849.5 | 5657.7 |  6852.6 | 6852.6 | 6852.6 | 6852.6 |
|   6 |       21 |       18 | 5819.4 | 6789.3 |  8223.1 | 8223.1 | 8223.1 | 8223.1 |